In [1]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import CompleteHist as ch
import Analysis
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
%matplotlib

Using matplotlib backend: TkAgg


In [3]:
run14 = Analysis.Run(np.load('2014_Raw.npy'), np.load('2014_Prune_strong.npy'), Analysis.bval2014, np.load('2014bf_new_prune_edit.npy'))
run11 = Analysis.Run(np.load('2011_Raw.npy'), np.load('2011_Prune_strong_flat.npy'), Analysis.bval2011, np.load('2011bf_new_prune_flat_edit.npy'))

soft11 = Analysis.Run(np.load('2011_Raw.npy'), np.load('2011_Prune_new_flat.npy'), Analysis.bval2011, np.load('2011bf_new_prune_flat_edit.npy'))
soft14 = Analysis.Run(np.load('2014_Raw.npy'), np.load('2014_Prune_new.npy'), Analysis.bval2014, np.load('2014bf_new_prune_edit.npy'))

i14 = run14.prune_data
i11 = run11.prune_data

s14 = soft14.prune_data
s11 = soft11.prune_data

In [4]:
se14 = np.load('2014se_new_sign.npy')
se11 = np.load('2011se_new_sign.npy')
#use this to plot slips:
#for k in se11[n][np.abs(se11[n][:,2] - i11[n].fits[j].delta_phi)<0.01]:
    #ax5.axvline(k[0],color='k',lw=2)

In [5]:
plt.rc('text', usetex=True) 

# Helvetica 
# set serif, too. Otherwise setting to times and then 
# Helvetica causes an error. 
font = {'family':'sans-serif','sans-serif':['Helvetica'], 
        'serif':['cm10']} 
preamble  = r""" 
               \usepackage{color} 
               \usepackage[tx]{sfmath} 
               \usepackage{helvet} 
            """ 


# Unicode for Tex 
#preamble =  r"""\usepackage[utf8]{inputenc}""" + preamble 
# inputenc should be set automatically 
plt.rcParams['text.latex.unicode']=True 

#print font, preamble 
plt.rc('font',**font) 
plt.rcParams['text.latex.preamble'] = preamble
mpl.rcParams.update({'font.size': 11})

In [6]:
from matplotlib.gridspec import GridSpec
gs = GridSpec(1000,1000)

def search(data, t):
    seq = data
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max < min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m

def t(y_min,y_max):
    diff = y_max-y_min
    p = float(diff)*0.35
    return y_min-p,y_max+p

def init_plot(i):
    i.plot()
    plt.xlim([min(i.x),max(i.x)])
    plt.ylim([min(i.y)-0.05,max(i.y)+0.05])
    
def stitch(i,x):
    i.fits[1].plot_select('b',['k','r'],[2,2],i.fits[1].pos_left,x)
    i.fits[0].plot_select('b',['r','#00ff00'],[2,2],x,i.fits[0].pos_right)
    
def set_axis(sweep,x_min,x_max):
    plt.xlim(x_min,x_max)
    x = np.linspace(x_min,x_max)
    yl,yr = sweep.fits[j].yValue(x)
    y_min = np.min([yl,yr])
    y_max = np.max([yl,yr])
    l,h = t(y_min,y_max)
    plt.ylim(l,h)
    plt.yticks([y_min,y_max],(round(y_min,1),round(y_max,1)))
    x_low, x_high = np.percentile(x,[25,75])
    plt.xticks([int(x_low),int(x_high)])


In [7]:
def newSE_count(run,se,n,j):
    temp = list()
    for k in se[n][np.abs(se[n][:,2] - (run.prune_data[n].fits[j].phase_r-run.prune_data[n].fits[j].phase_l)/np.pi%2)<0.001]:
        temp.append(k)
    return temp


In [8]:
def plotGrid(run,se,start,fit_list,k,x_low,x_high,colspan,rowspan,xy=(-0.05, 1.02),lw=1.2,linew=2):
    plt.subplot2grid((100,100), start, colspan=colspan, rowspan=rowspan)
    j = fit_list[k][1]
    n = fit_list[k][0]
    run.prune_data[n].plot_fit(j,linewidth=linew)
    axes(run.prune_data[n],j,x_low,x_high)
    for k in se[n][np.abs(se[n][:,2] - (run.prune_data[n].fits[j].phase_r-run.prune_data[n].fits[j].phase_l)/np.pi%2)<0.001]:
        plt.axvline(k[0],color='k',lw=lw)
    plt.annotate('{:.4f} T, $\Delta\\theta=$ {:.3f}'.format(run.prune_data[n].b_val,round(((run.prune_data[n].fits[j].phase_r-run.prune_data[n].fits[j].phase_l)/np.pi)%2,3)), xy, xycoords="axes fraction")
        

def axes(sweep,j,x_min,x_max):
    plt.xlim(x_min,x_max)
    x = np.linspace(x_min,x_max)
    yl,yr = sweep.fits[j].yValue(x)
    y_min = np.min([yl,yr])
    y_max = np.max([yl,yr])
    l,h = t(y_min,y_max)
    plt.ylim(l,h)
    plt.yticks([y_min,y_max],(round(y_min,1),round(y_max,1)))
    x_low, x_high = np.percentile(x,[25,75])
    plt.xticks([int(x_low),int(x_high)])
    ax = plt.gca()
    ax.tick_params('y',pad=2)
    ax.tick_params('x',pad=3)
    
    
def newplotGrid(bounds,colspan,rowspan,xy=(-0.05, 1.02)):
    run,se,fit_list,k,x_low,x_high = bounds
    j = fit_list[k][1]
    n = fit_list[k][0]
    run.prune_data[n].plot_fit(j)
    axes(run.prune_data[n],j,x_low,x_high)
    for k in se[n][np.abs(se[n][:,2] - (run.prune_data[n].fits[j].phase_r-run.prune_data[n].fits[j].phase_l)/np.pi%2)<0.001]:
        plt.axvline(k[0],color='k',lw=2)
    plt.annotate('{} T'.format(run.prune_data[n].b_val), xy, xycoords="axes fraction")


In [9]:
pi4_14 = [[0,0],[0,2],[0,3],[2,1],[2,2],[3,3],[3,4],[4,1],[5,0],[7,2],[8,0],[8,3],[9,0]]
pi2_14 = [[3,2],[3,1]] 
pi34_14 = [[2,0],[2,3],[3,0],[8,1]]
pi_14 = [[0,1],[4,0],[5,1],[6,0],[7,1],[8,2],[9,1],[10,0],[11,0],[13,0],[13,1]]
pi74_14 = [[5,2]]
#------------------------------------------
pi4_11 = [[0,0],[5,0],[9,1],[11,0],[14,0],[19,1]]
pi2_11 = [[2,1],[4,0],[12,1],[15,0],[19,0]]
pi34_11 = [[0,1],[1,1],[10,0],[13,0]]
pi_11 = [[1,0],[2,0],[3,0],[4,1],[5,1],[6,0],[7,0],[8,0],[9,2],[11,1],[12,0],[14,1],[15,1],[16,0],[17,0],[19,2],[21,0]]
#No pi7/4 for 2011

In [10]:
pi_11_bounds = [[soft11,se11,pi_11,0,-26.34,-18.02],[run11,se11,pi_11,1,-38.84,-29.39],
                [run11,se11,pi_11,2,-41.13,-29.99],[soft11,se11,pi_11,3,-42.59,-31.06],
                [run11,se11,pi_11,4,-41.27,-30.44],[soft11,se11,pi_11,5,-41,-30],
                [soft11,se11,pi_11,6,-40,-29.82],[run11,se11,pi_11,7,-39.23,-28.47],
                [run11,se11,pi_11,8,-39.9,-29.56],[run11,se11,pi_11,9,-38.2,-29.144],
                [run11,se11,pi_11,10,-42.07,-31.46],[run11,se11,pi_11,11,-41.87,-29.9],
                [run11,se11,pi_11,12,-41.29,-30.3],[run11,se11,pi_11,13,-40.29,-30.86],
                [run11,se11,pi_11,14,-41.36,-32.15],[run11,se11,pi_11,15,-41.33,-31.54],
                [run11,se11,pi_11,16,-40.7672,-30.3]]

pi_14_bounds = [[soft14,se14,pi_14,0,-19.48,-9.83],[soft14,se14,pi_14,1,-18.18,-7.05],
       [run14,se14,pi_14,2,-21.14,-8.78],[run14,se14,pi_14,3,-16.54,-9],
       [soft14,se14,pi_14,4,-17.8,-10.38],[run14,se14,pi_14,5,-15.18,-7.22],
       [soft14,se14,pi_14,6,-17.8,-9.72],[soft14,se14,pi_14,7,-20.02,-9.20],
       [run14,se14,pi_14,8,-20.4,-8.65],[run14,se14,pi_14,9,-19.32,-7.76],
       [run14,se14,pi_14,10,-27.51,-17.93]]

In [11]:
pi_new = list()

pi_bounds = list(pi_11_bounds)
pi_bounds.extend(pi_14_bounds)
pi_bounds = np.array(pi_bounds)


for j in [14,13,12,11,8,7,6,5,4,3,2,0,23,21,20]:
    pi_new.append(pi_bounds[j])


pi_new[0][-2],pi_new[0][-1] = -42.01,-29.714
pi_new[1][-2],pi_new[1][-1] = -43.739,-30.7454
pi_new[2][-2],pi_new[2][-1] = -45.9674,-30.1864
pi_new[3][-2],pi_new[3][-1] = -42.0237, -29.9358
pi_new[4][-2],pi_new[4][-1] = -41.2647, -29.5113
pi_new[5][-2],pi_new[5][-1] = -40.6632, -28.4236
pi_new[6][-2],pi_new[6][-1] = -40, -29.8989
pi_new[7][-2],pi_new[7][-1] = -45.3157, -29.8903
pi_new[8][-2],pi_new[8][-1] = -43.3091, -30.3461
pi_new[9][-2],pi_new[9][-1] = -42.7017, -31.0218
pi_new[10][-2],pi_new[10][-1] = -41.9687, -29.6581
pi_new[11][-2],pi_new[11][-1] = -26.4479, -17.6429
pi_new[12][-2],pi_new[12][-1] = -19.7687, -7.3317
pi_new[13][-2],pi_new[13][-1] = -21.0172, -9.7475
pi_new[14][-2],pi_new[14][-1] = -17.1242, -5.575

In [15]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 5
width = 3
row_extra = 4
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(len(pi_new)):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_new[i])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 6:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        #ax = plt.gca()
        #ax.yaxis.set_label_coords(-.17,1.1)
    if i == 13:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)    

In [22]:
fig.savefig('Pi v2',dpi=1000)

In [16]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 5
width = 3
row_extra = 4
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(len(pi_new)):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_new[-i-1])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 6:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        #ax = plt.gca()
        #ax.yaxis.set_label_coords(-.17,1.1)
    if i == 13:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)    

In [24]:
fig.savefig('Pi v2b',dpi=1000)

In [37]:
def find_edge(a):
    for i in a:
        k = i[3]
        n = i[2][k][0]
        j = i[2][k][1]
        plt.figure()
        yield i[0].prune_data[n].plot_fit(j)

In [38]:
a = find_edge(pi_sup)

NameError: name 'pi_sup' is not defined

In [12]:
pi_sup = list()

pi_bounds = list(pi_11_bounds)
pi_bounds.extend(pi_14_bounds)
pi_bounds = np.array(pi_bounds)


for j in [16,15,10,9,1,27,26,25,24,22,19,18,17]:
    pi_sup.append(pi_bounds[j])


pi_sup[0][-2],pi_sup[0][-1] = -42.9495,-30.1496
pi_sup[1][-2],pi_sup[1][-1] = -42.3122, -30.0638
pi_sup[2][-2],pi_sup[2][-1] = -46.7153, -30.7536
pi_sup[3][-2],pi_sup[3][-1] = -43.7484, -29.2676
pi_sup[4][-2],pi_sup[4][-1] = -38.8846, -29.3708
pi_sup[5][-2],pi_sup[5][-1] = -27.4837, -17.0424
pi_sup[6][-2],pi_sup[6][-1] = -19.7555, -7.7148
pi_sup[7][-2],pi_sup[7][-1] = -20.5392, -8.4924
pi_sup[8][-2],pi_sup[8][-1] = -20.2131, -8.3886
pi_sup[9][-2],pi_sup[9][-1] = -15.3165, -7.1523
pi_sup[10][-2],pi_sup[10][-1] = -20.9327, -12.2644
pi_sup[11][-2],pi_sup[11][-1] = -20.6114, -7.0573
pi_sup[12][-2],pi_sup[12][-1] = -19.4212, -8.5128
#pi_sup[13][-2],pi_sup[13][-1] = 
#pi_sup[14][-2],pi_sup[14][-1] = 

In [213]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 5
width = 3
row_extra = 4
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(len(pi_sup)):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_sup[i])
    if i == 12:
        i += 1
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 6:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        #ax = plt.gca()
        #ax.yaxis.set_label_coords(-.17,1.1)
      

In [63]:
fig.savefig('Supplemental Pi v1a',dpi=1000)

In [214]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 5
width = 3
row_extra = 4
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(len(pi_sup)):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_sup[-i-1])
    if i == 12:
        i += 1
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
        plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        continue
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 6:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        #ax = plt.gca()
        #ax.yaxis.set_label_coords(-.17,1.1)
      

In [65]:
fig.savefig('Supplemental Pi v1b',dpi=1000)

In [56]:
i14[4].plot_fit(0)

In [57]:
soft14.prune_data[4].plot_fit(0)

In [66]:
se11[7]

array([[-34.666045  ,   5.5325    ,   0.90381885,   0.93540052],
       [-34.49799   ,   5.5325    ,   0.90381885,   0.93540052],
       [-33.65      ,   5.5325    ,   0.90381885,   0.93540052]])

In [76]:
run11.bvals.index(5.5265)

19

In [64]:
plt.figure(figsize=(9,11))
l = [0,5,6,15,17,19,21]
for i in range(7):
    plt.subplot(7,1,i+1)
    s11[l[i]].plot_fit(0)

In [58]:
j = 0

In [59]:
plt.clf()
i11[j].plot_fit(0)
print(j)
j += 1

21


In [79]:
fig = plt.figure(figsize=(9,11))
count = 6
#data_type = i11 #version A
data_type = s11 #Version B

plt.subplot(count,1,1)
n = 0
j = 1
data_type[n].plot_fit(j, False)
axes(data_type[n],j,-42.9,-23.4912)

plt.subplot(count,1,2)
n = 5
j = 1
data_type[n].plot_fit(j, False)
axes(data_type[n],j,-44.44,-20)

plt.subplot(count,1,3)
n = 6
j = 0
s11[n].plot_fit(j, False)
axes(data_type[n],j,-45,-23.87)

plt.subplot(count,1,4)
n = 17
j = 0
data_type[n].plot_fit(j, False)
axes(data_type[n],j,-45.798,-20)

plt.subplot(count,1,5)
n = 19
j = 2
data_type[n].plot_fit(j, False)
axes(data_type[n],j,-42.4,-17.465)

plt.subplot(count,1,6)
n = 21
j = 0
data_type[n].plot_fit(j, False)
axes(data_type[n],j,-42.05,-20.52)

In [80]:
fig.savefig('Long Pi v1b',dpi=1000)

In [14]:
data_type = i11 #version A
#data_type = s11 #Version B


xy=(-0.023, 1.02)
lw = 2.0 #slip lines
linew = 2.5 #fit lines
height = 3
width = 1
row_extra = 8
row_len = (100-row_extra*(height-1))/height
col_extra = 0
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (2*1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

out = list()
for i in range(3):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    plt.subplot2grid((100,100), current, colspan=col_len, rowspan=row_len)


    if i == 0:
        n = 17
        j = 0
        data_type[n].plot()
        data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-41.35,-31.249)
        axes(data_type[n],j,-42.01,-21.37)
        plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
        plt.xticks([-40,-30])
        
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

    if i == 1:
        n = 19
        j = 2
        data_type[n].plot()
        data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-41.98,-31.72)
        data_type[n].fits[0].plot_select('b',['k','r'],[linew,linew],-28.17,-17.465)        
        axes(data_type[n],j,-42.4,-17.465)
        plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
        plt.xticks([-40,-30,-20])
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=18)
        
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[0].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

    if i == 2:
        n = 21
        j = 0
        data_type[n].plot()
        data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-40.68,-30.77)
        axes(data_type[n],j,-42.05,-21.52)
        plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
        plt.xticks([-40,-30])
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        

In [134]:
fig.savefig('Long Pi v2',dpi=1000)

In [102]:
data_type[n].fits[j].plot()

In [12]:
data_type = i11 #version A
#data_type = s11 #Version B



lw = 2.0 #slip lines
linew = 2.5 #fit lines
long_slips = 3
height = 8
width = 3
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

out = list()

xy=(-0.008, 1.02)
for i in range(3):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    plt.subplot2grid((100,100), current, colspan=3*col_len+2*col_extra, rowspan=row_len)


    if i == 0:
        n = 17
        j = 0
        data_type[n].plot()
        data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-41.35,-31.249)
        axes(data_type[n],j,-42.01,-22.83)
        plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
        plt.xticks([-40,-30])
        
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

    if i == 1:
        n = 19
        j = 2
        data_type[n].plot()
        data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-41.98,-31.72)
        data_type[n].fits[0].plot_select('b',['k','r'],[linew,linew],-28.17,-17.465)        
        axes(data_type[n],j,-42.4,-17.465)
        plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
        plt.xticks([-40,-30,-20])
        #plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=18)
        
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[0].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

    if i == 2:
        n = 21
        j = 0
        data_type[n].plot()
        data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-40.68,-30.77)
        axes(data_type[n],j,-42.05,-22.61)
        plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
        plt.xticks([-40,-30])
        #plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

xy=(-0.023, 1.02)
for i in range(long_slips*3,len(pi_new)+long_slips*3):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_new[-i+long_slips*3-1])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == long_slips*3+3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == long_slips*3+13:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)    

In [172]:
fig.savefig('Pi v3a',dpi=1000)

In [13]:
data_type = i11 #version A
#data_type = s11 #Version B



lw = 2.0 #slip lines
linew = 2.5 #fit lines
long_slips = 2
height = 7
width = 3
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

out = list()

xy=(-0.008, 1.02)
for i in range(long_slips):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    plt.subplot2grid((100,100), current, colspan=3*col_len+2*col_extra, rowspan=row_len)


    #if i == 0:
    #    n = 17
    #    j = 0
    #    data_type[n].plot()
    #    data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-41.35,-31.249)
    #    axes(data_type[n],j,-42.01,-22.83)
    #    plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
    #    plt.xticks([-40,-30])
    #    
    #    for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
    #        plt.axvline(k[0],color='k',lw=lw)

    if i == 0:
        n = 19
        j = 2
        data_type[n].plot()
        data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-41.98,-31.72)
        data_type[n].fits[0].plot_select('b',['k','r'],[linew,linew],-28.17,-17.465)        
        axes(data_type[n],j,-42.4,-17.465)
        plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
        plt.xticks([-40,-30,-20])
        #plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=18)
        
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[0].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

    if i == 1:
        n = 21
        j = 0
        data_type[n].plot()
        data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-40.68,-30.77)
        axes(data_type[n],j,-42.05,-22.61)
        plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
        plt.xticks([-40,-30])
        #plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

xy=(-0.023, 1.02)
for i in range(long_slips*3,len(pi_new)+long_slips*3):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_new[-i+long_slips*3-1])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == long_slips*3+3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == long_slips*3+13:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)    

In [175]:
fig.savefig('Pi v3b',dpi=1000)

In [205]:
data_type = i11 #version A
#data_type = s11 #Version B



lw = 2.0 #slip lines
linew = 2.5 #fit lines
long_slips = 2
height = 7
width = 3
row_extra = 3
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

out = list()

xy=(-0.008, 1.02)
for i in range(long_slips):
    current = (i%height*(row_extra+row_len),i/height*(col_extra+col_len))
    plt.subplot2grid((100,100), current, colspan=3*col_len+2*col_extra, rowspan=row_len)


    #if i == 0:
    #    n = 17
    #    j = 0
    #    data_type[n].plot()
    #    data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-41.35,-31.249)
    #    axes(data_type[n],j,-42.01,-22.83)
    #    plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
    #    plt.xticks([-40,-30])
    #    out.append(newSE_count(hold[0],hold[1],hold[2][hold[3]][0],hold[2][hold[3]][1]))
    #    for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
    #        plt.axvline(k[0],color='k',lw=lw)

    if i == 0:
        n = 19
        j = 2
        data_type[n].plot()
        data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-41.98,-31.72)       
        axes(data_type[n],j,-42.4,-24.8)
        plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
        plt.xticks([-40,-30])
        #plt.ylabel('$R_D$ $(k\Omega)$',labelpad=3,size=18)
        out.append(newSE_count(run11,se11,n,j))
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)


    if i == 1:
        n = 21
        j = 0
        data_type[n].plot()
        data_type[n].fits[j].plot_select('b',['k','r'],[linew,linew],-40.68,-30.77)
        axes(data_type[n],j,-42.05,-22.61)
        plt.annotate('{:.4f} T'.format(data_type[n].b_val), xy, xycoords="axes fraction")
        plt.xticks([-40,-30])
        #plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        out.append(newSE_count(run11,se11,n,j))
        for k in se11[n][np.abs(se11[n][:,2] -  (2-data_type[n].fits[j].delta_phi))<0.01]:
            plt.axvline(k[0],color='k',lw=lw)

xy=(-0.023, 1.02)
for i in range(long_slips*3,len(pi_new)+long_slips*3):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_new[-i+long_slips*3-1])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    out.append(newSE_count(hold[0],hold[1],hold[2][hold[3]][0],hold[2][hold[3]][1]))
    if i == long_slips*3+3:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == long_slips*3+13:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)    

In [184]:
fig.savefig('Pi v3c',dpi=1000)

In [206]:
out

[[array([-35.38952   ,   5.5265    ,   0.96946642,   0.95348837])],
 [array([-34.89967   ,   5.5255    ,   0.90145079,   0.92      ])],
 [array([-15.1785    ,   5.5815    ,   0.89620572,   1.        ])],
 [array([-14.168     ,   5.581     ,   0.99735913,   1.        ])],
 [array([-13.838535 ,   5.58     ,   1.0219144,   1.       ])],
 [array([-20.059545  ,   5.5355    ,   1.15402782,   0.47787611])],
 [array([-35.49629   ,   5.5345    ,   0.94090891,   0.84615385]),
  array([-33.935365  ,   5.5345    ,   0.94090891,   0.84615385]),
  array([-33.72232   ,   5.5345    ,   0.94090891,   0.84615385])],
 [array([-41.42871   ,   5.534     ,   0.9524676 ,   0.85202864]),
  array([-41.257855  ,   5.534     ,   0.9524676 ,   0.85202864]),
  array([-36.534725  ,   5.534     ,   0.9524676 ,   0.85202864]),
  array([-36.01296   ,   5.534     ,   0.9524676 ,   0.85202864]),
  array([-35.657465  ,   5.534     ,   0.9524676 ,   0.85202864])],
 [array([-35.21645   ,   5.5335    ,   0.98255864,   0.852

In [208]:
np.save('Pruned 1 Fig 3_12_2019', out)

In [55]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 4
width = 3
row_extra = 4
row_len = (100-row_extra*(height-1))/height
col_extra = 4
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
out = list()
for i in range(len(pi_sup)-1):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_sup[-i-2])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    out.append(newSE_count(hold[0],hold[1],hold[2][hold[3]][0],hold[2][hold[3]][1]))
    if i == 6:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 10:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
      

In [223]:
fig.savefig('Supplemental Pi v2',dpi=1000)

In [225]:
out

[[array([-14.4746    ,   5.5825    ,   1.02110047,   1.        ]),
  array([-7.651     ,  5.5825    ,  1.02110047,  1.        ]),
  array([-7.43      ,  5.5825    ,  1.02110047,  1.        ])],
 [array([-14.2383    ,   5.582     ,   0.99611736,   1.        ])],
 [array([-13.8221   ,   5.5805   ,   0.9816624,   1.       ])],
 [array([-13.9362    ,   5.5795    ,   0.95692798,   1.        ])],
 [array([-13.7099    ,   5.579     ,   0.94285897,   1.        ])],
 [array([-15.348325  ,   5.578     ,   0.91429435,   1.        ]),
  array([-14.955145  ,   5.578     ,   0.91429435,   1.        ]),
  array([-12.94725   ,   5.578     ,   0.91429435,   1.        ])],
 [array([-21.7499    ,   5.578     ,   0.94609601,   1.        ])],
 [array([-35.588805  ,   5.535     ,   0.94226665,   0.83892617])],
 [array([-35.00868   ,   5.5305    ,   0.91014898,   0.89013453]),
  array([-34.85053   ,   5.5305    ,   0.91014898,   0.89013453]),
  array([-33.961335  ,   5.5305    ,   0.91014898,   0.89013453]),

In [226]:
np.save('Pruned 1 Sup 3_12_2019', out)

In [231]:
run14.bvals.index(5.5815)

6

In [232]:
se14[6]

array([[-15.1785    ,   5.5815    ,   0.89620572,   1.        ]])

In [239]:
(i14[6].fits[0].phase_r-i14[6].fits[0].phase_l)/(np.pi)%2

0.89620572379399155

In [14]:
pi_new = list()

pi_bounds = list(pi_11_bounds)
pi_bounds.extend(pi_14_bounds)
pi_bounds = np.array(pi_bounds)


for j in [15,14,13,12,11,8,7,6,5,4,3,2,23,21,20]:
    pi_new.append(pi_bounds[j])

pi_new[0][-2],pi_sup[0][-1] = -42.3122, -30.0638
pi_new[1][-2],pi_new[1][-1] = -42.01,-29.714
pi_new[2][-2],pi_new[2][-1] = -43.739,-30.7454
pi_new[3][-2],pi_new[3][-1] = -45.9674,-30.1864
pi_new[4][-2],pi_new[4][-1] = -42.0237, -29.9358
pi_new[5][-2],pi_new[5][-1] = -41.2647, -29.5113
pi_new[6][-2],pi_new[6][-1] = -40.6632, -28.4236
pi_new[7][-2],pi_new[7][-1] = -40, -29.8989
pi_new[8][-2],pi_new[8][-1] = -45.3157, -29.8903
pi_new[9][-2],pi_new[9][-1] = -43.3091, -30.3461
pi_new[10][-2],pi_new[10][-1] = -42.7017, -31.0218
pi_new[11][-2],pi_new[11][-1] = -41.9687, -29.6581
pi_new[12][-2],pi_new[12][-1] = -19.7687, -7.3317
pi_new[13][-2],pi_new[13][-1] = -21.0172, -9.7475
pi_new[14][-2],pi_new[14][-1] = -17.1242, -5.575

In [15]:
pi_sup = list()

pi_bounds = list(pi_11_bounds)
pi_bounds.extend(pi_14_bounds)
pi_bounds = np.array(pi_bounds)


for j in [16,10,9,1,27,26,25,24,22,19,18,17]:
    pi_sup.append(pi_bounds[j])


pi_sup[0][-2],pi_sup[0][-1] = -42.9495,-30.1496

pi_sup[1][-2],pi_sup[1][-1] = -46.7153, -30.7536
pi_sup[2][-2],pi_sup[2][-1] = -43.7484, -29.2676
pi_sup[3][-2],pi_sup[3][-1] = -38.8846, -29.3708
pi_sup[4][-2],pi_sup[4][-1] = -27.4837, -17.0424
pi_sup[5][-2],pi_sup[5][-1] = -19.7555, -7.7148
pi_sup[6][-2],pi_sup[6][-1] = -20.5392, -8.4924
pi_sup[7][-2],pi_sup[7][-1] = -20.2131, -8.3886
pi_sup[8][-2],pi_sup[8][-1] = -15.3165, -7.1523
pi_sup[9][-2],pi_sup[9][-1] = -20.9327, -12.2644
pi_sup[10][-2],pi_sup[10][-1] = -20.6114, -7.0573
pi_sup[11][-2],pi_sup[11][-1] = -19.4212, -8.5128
#pi_sup[13][-2],pi_sup[13][-1] = 
#pi_sup[14][-2],pi_sup[14][-1] = 

In [27]:
#This is stupid but I'm commited to the dumb list

pi = list()
pi.append(pi_sup[0])
pi.extend(pi_new[0:5])
pi.extend(pi_sup[1:3])
pi.extend(pi_new[5:12])
pi.extend(pi_sup[3:8])
pi.append(pi_new[-3])
pi.append(pi_sup[8])
pi.extend(pi_new[-2:])
pi.extend(pi_sup[-3:])

pi = pi[::-1]


In [28]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 4
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(8):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi[i])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 4:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 6:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05,-0.23)

In [19]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 4
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(8):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_new[-i-1])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 4:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 6:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05,-0.23)

In [20]:
fig.savefig('52_1')

In [22]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 4
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(7):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_new[-i-8])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 4:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 6:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05,-0.23)

In [35]:
fig.savefig('52_2')

In [43]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 4
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
out = list()
for i in range(8):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_sup[-i-2])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    out.append(newSE_count(hold[0],hold[1],hold[2][hold[3]][0],hold[2][hold[3]][1]))
    if i == 4:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 6:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05,-0.23)
      

In [44]:
fig.savefig('52_3')

In [52]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 2
width = 2
bot = 5
row_extra = 12
row_len = (100-row_extra*(height-1)-bot)/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))
out = list()
for i in range(4):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi_sup[-i-10])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    out.append(newSE_count(hold[0],hold[1],hold[2][hold[3]][0],hold[2][hold[3]][1]))
    if i == 2:
        plt.ylabel('$R_D$ $(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 2:
        plt.xlabel('$V_P$ $(mV)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05,-0.23)
      

In [53]:
fig.savefig('52_4')

In [54]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 4
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(8):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi[i])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 4:
        plt.ylabel('$R_D(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 6:
        plt.xlabel('$V_P(mV)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05,-0.23)

In [55]:
fig.savefig('52_1')

In [56]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 4
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(8):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi[i+8])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 4:
        plt.ylabel('$R_D(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 6:
        plt.xlabel('$V_P(mV)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05,-0.23)

In [57]:
fig.savefig('52_2')

In [58]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 4
width = 2
row_extra = 6
row_len = (100-row_extra*(height-1))/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(8):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    hold = list(pi[i+8+8])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 4:
        plt.ylabel('$R_D(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.17,1.1)
    if i == 6:
        plt.xlabel('$V_P(mV)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.xaxis.set_label_coords(1.05,-0.23)

In [59]:
fig.savefig('52_3')

In [60]:
lw = 1.6 #slip lines
linew = 2.5 #fit lines
xy=(-0.023, 1.02)
height = 2
width = 2
row_extra = 13
bot = 5
row_len = (100-row_extra*(height-1)-bot)/height
col_extra = 6
col_len = (100-col_extra*(width-1))/width
fig = plt.figure(figsize = (1.5*2.42/(col_len/100.),1.5/(row_len/100.)))

for i in range(3):
    current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len))
    if i == 2:
        current = (i/width*(row_extra+row_len),i%width*(col_extra+col_len)+int((col_extra+col_len)/2))
    hold = list(pi[i+8+8+8])
    plotGrid(hold[0],hold[1],current,hold[2],hold[3],hold[4],hold[5],col_len,row_len,xy=xy,lw=lw,linew=linew)
    if i == 2:
        plt.ylabel('$R_D(k\Omega)$',labelpad=4,size=18)
        ax = plt.gca()
        ax.yaxis.set_label_coords(-.72,1.1)
    if i == 2:
        plt.xlabel('$V_P(mV)$',labelpad=4,size=18)
        #ax = plt.gca()
        #ax.xaxis.set_label_coords(1.05,-0.23)

In [61]:
fig.savefig('52_4')